## Goal:
The goal of this script is to be able to gain the values of the higest and lowest of the metadata for a user to be able to play so that they can understand what the data means

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import argparse
import logging
import spotipy
import pandas as pd
import numpy as np
import requests
from spotipy import oauth2
import re

import credentials

In [8]:
# Gain access to mainupation
SCOPE = ('user-read-recently-played,user-top-read,user-library-read,user-read-currently-playing,playlist-read-private,playlist-modify-private,playlist-modify-public,user-read-email,user-modify-playback-state,user-read-private,user-read-playback-state')
sp_oauth = oauth2.SpotifyOAuth(credentials.SPOTIPY_CLIENT_ID,credentials.SPOTIPY_CLIENT_SECRET, credentials.SPOTIPY_REDIRECT_URI ,scope=SCOPE )

#click "Accept" in your browser when the auth window pops up
code = sp_oauth.get_auth_response(open_browser=True)
token = sp_oauth.get_access_token(code)
refresh_token = token['refresh_token']
sp = spotipy.Spotify(auth=token['access_token'])
username = sp.current_user()['id']

/var/folders/pz/lc5f5cgd1m5_ghmjddlz8r600000gn/T/ipykernel_80970/3544539949.py:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = sp_oauth.get_access_token(code)


In [25]:
ranges = ['short_term', 'medium_term', 'long_term']

top_tracks = []

for sp_range in ranges:
    results = sp.current_user_top_tracks(time_range=sp_range, limit=50)
    for i in range(len(results)):
        top_tracks.append(results['items'][i]['uri'])

In [26]:
print(len(top_tracks))

21


In [32]:
# Get audio features
audio_features = sp.audio_features(top_tracks)

In [33]:
df = pd.DataFrame(audio_features)

In [35]:
# get index of row with largest danceabilty value
idx = df['danceability'].idxmax()
uri = df.loc[idx, 'uri']

In [38]:
uri = df.loc[df['danceability'].idxmax(), 'uri']

In [39]:
print(uri)

spotify:track:3hX0cncUUZ3A6cx9w1dW4w


In [41]:
songs = {'danceability': [df.loc[df['danceability'].idxmax(), 'uri'], df.loc[df['danceability'].idxmin(), 'uri']], 
        'energy': [df.loc[df['energy'].idxmax(), 'uri'], df.loc[df['energy'].idxmin(), 'uri']], 
         'valence': [df.loc[df['valence'].idxmax(), 'uri'], df.loc[df['valence'].idxmin(), 'uri']],
         'instrumentalness': [df.loc[df['instrumentalness'].idxmax(), 'uri'], df.loc[df['instrumentalness'].idxmin(), 'uri']],
         'tempo': [df.loc[df['tempo'].idxmax(), 'uri'], df.loc[df['tempo'].idxmin(), 'uri']]}

In [42]:
def get_top_features(sp):
    ranges = ['short_term', 'medium_term', 'long_term']

    top_tracks = []

    for sp_range in ranges:
        results = sp.current_user_top_tracks(time_range=sp_range, limit=50)
        for i in range(len(results)):
            top_tracks.append(results['items'][i]['uri'])
    audio_features = sp.audio_features(top_tracks)
    df = pd.DataFrame(audio_features)
    songs = {'danceability': [df.loc[df['danceability'].idxmax(), 'uri'], df.loc[df['danceability'].idxmin(), 'uri']], 
        'energy': [df.loc[df['energy'].idxmax(), 'uri'], df.loc[df['energy'].idxmin(), 'uri']], 
         'valence': [df.loc[df['valence'].idxmax(), 'uri'], df.loc[df['valence'].idxmin(), 'uri']],
         'instrumentalness': [df.loc[df['instrumentalness'].idxmax(), 'uri'], df.loc[df['instrumentalness'].idxmin(), 'uri']],
         'tempo': [df.loc[df['tempo'].idxmax(), 'uri'], df.loc[df['tempo'].idxmin(), 'uri']]}
    return songs